In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:18.0000 R:18.0000 rate:0.0360 aloss:0.6851 eloss:2.9327 aloss2:1.4346 exploreP:0.9982
Episode:1 meanR:21.0000 R:24.0000 rate:0.0480 aloss:0.6857 eloss:2.8895 aloss2:1.4715 exploreP:0.9959
Episode:2 meanR:25.0000 R:33.0000 rate:0.0660 aloss:0.6895 eloss:2.9255 aloss2:1.4417 exploreP:0.9926
Episode:3 meanR:22.2500 R:14.0000 rate:0.0280 aloss:0.6828 eloss:2.9081 aloss2:1.4451 exploreP:0.9912
Episode:4 meanR:31.2000 R:67.0000 rate:0.1340 aloss:0.6864 eloss:2.9374 aloss2:1.4400 exploreP:0.9847
Episode:5 meanR:28.3333 R:14.0000 rate:0.0280 aloss:0.6881 eloss:2.9032 aloss2:1.4722 exploreP:0.9833
Episode:6 meanR:25.5714 R:9.0000 rate:0.0180 aloss:0.6817 eloss:2.9824 aloss2:1.3975 exploreP:0.9824
Episode:7 meanR:26.0000 R:29.0000 rate:0.0580 aloss:0.6896 eloss:2.9648 aloss2:1.4208 exploreP:0.9796
Episode:8 meanR:28.3333 R:47.0000 rate:0.0940 aloss:0.6894 eloss:2.8697 aloss2:1.5009 exploreP:0.9751
Episode:9 meanR:32.6000 R:71.0000 rate:0.1420 aloss:0.6872 eloss:2.8982 aloss2:1.48

Episode:80 meanR:27.5556 R:58.0000 rate:0.1160 aloss:0.6864 eloss:1.4271 aloss2:3.1666 exploreP:0.8020
Episode:81 meanR:27.4390 R:18.0000 rate:0.0360 aloss:0.6903 eloss:1.4081 aloss2:3.1632 exploreP:0.8005
Episode:82 meanR:27.6506 R:45.0000 rate:0.0900 aloss:0.6828 eloss:1.3660 aloss2:3.3294 exploreP:0.7970
Episode:83 meanR:27.5952 R:23.0000 rate:0.0460 aloss:0.6842 eloss:1.4229 aloss2:3.2037 exploreP:0.7952
Episode:84 meanR:28.0353 R:65.0000 rate:0.1300 aloss:0.6875 eloss:1.3443 aloss2:3.3032 exploreP:0.7901
Episode:85 meanR:27.9186 R:18.0000 rate:0.0360 aloss:0.6799 eloss:1.3169 aloss2:3.3713 exploreP:0.7887
Episode:86 meanR:27.7471 R:13.0000 rate:0.0260 aloss:0.6767 eloss:1.2508 aloss2:3.4648 exploreP:0.7877
Episode:87 meanR:27.7955 R:32.0000 rate:0.0640 aloss:0.6842 eloss:1.2738 aloss2:3.3972 exploreP:0.7852
Episode:88 meanR:27.9775 R:44.0000 rate:0.0880 aloss:0.6784 eloss:1.2143 aloss2:3.5065 exploreP:0.7818
Episode:89 meanR:28.3222 R:59.0000 rate:0.1180 aloss:0.6853 eloss:1.2621 

Episode:159 meanR:39.9100 R:101.0000 rate:0.2020 aloss:0.7135 eloss:-0.6416 aloss2:9.1765 exploreP:0.5774
Episode:160 meanR:40.7300 R:105.0000 rate:0.2100 aloss:0.7350 eloss:-0.8097 aloss2:9.4793 exploreP:0.5715
Episode:161 meanR:42.4000 R:182.0000 rate:0.3640 aloss:0.7217 eloss:-1.0158 aloss2:9.5514 exploreP:0.5614
Episode:162 meanR:42.7500 R:61.0000 rate:0.1220 aloss:0.7314 eloss:-1.1216 aloss2:10.1187 exploreP:0.5580
Episode:163 meanR:43.6100 R:116.0000 rate:0.2320 aloss:0.7303 eloss:-1.1511 aloss2:10.2275 exploreP:0.5517
Episode:164 meanR:43.3600 R:30.0000 rate:0.0600 aloss:0.7287 eloss:-1.2527 aloss2:10.3156 exploreP:0.5501
Episode:165 meanR:43.0900 R:24.0000 rate:0.0480 aloss:0.7313 eloss:-0.6203 aloss2:11.2485 exploreP:0.5488
Episode:166 meanR:43.7100 R:96.0000 rate:0.1920 aloss:0.7391 eloss:-1.2495 aloss2:10.5747 exploreP:0.5436
Episode:167 meanR:43.9600 R:61.0000 rate:0.1220 aloss:0.7237 eloss:-1.2929 aloss2:10.8254 exploreP:0.5404
Episode:168 meanR:44.7900 R:120.0000 rate:0.2

Episode:237 meanR:63.6900 R:67.0000 rate:0.1340 aloss:0.7218 eloss:-3.4639 aloss2:22.8042 exploreP:0.3401
Episode:238 meanR:63.8600 R:51.0000 rate:0.1020 aloss:0.7728 eloss:-3.5029 aloss2:22.8441 exploreP:0.3385
Episode:239 meanR:64.5100 R:78.0000 rate:0.1560 aloss:0.7184 eloss:-3.7324 aloss2:22.3475 exploreP:0.3359
Episode:240 meanR:65.1100 R:80.0000 rate:0.1600 aloss:0.7238 eloss:-3.5811 aloss2:22.8244 exploreP:0.3333
Episode:241 meanR:65.1700 R:45.0000 rate:0.0900 aloss:0.7392 eloss:-3.4627 aloss2:23.2019 exploreP:0.3319
Episode:242 meanR:64.9200 R:46.0000 rate:0.0920 aloss:0.7246 eloss:-3.9386 aloss2:22.7701 exploreP:0.3304
Episode:243 meanR:65.0500 R:24.0000 rate:0.0480 aloss:0.6624 eloss:-3.5180 aloss2:24.2753 exploreP:0.3296
Episode:244 meanR:64.7800 R:46.0000 rate:0.0920 aloss:0.7213 eloss:-2.5303 aloss2:23.6390 exploreP:0.3281
Episode:245 meanR:65.1700 R:54.0000 rate:0.1080 aloss:0.7104 eloss:-3.8333 aloss2:23.5662 exploreP:0.3264
Episode:246 meanR:65.6500 R:123.0000 rate:0.24

Episode:315 meanR:57.1200 R:54.0000 rate:0.1080 aloss:0.7005 eloss:-5.1963 aloss2:29.5248 exploreP:0.2284
Episode:316 meanR:57.0800 R:39.0000 rate:0.0780 aloss:0.7092 eloss:-2.5922 aloss2:30.9113 exploreP:0.2276
Episode:317 meanR:57.1700 R:32.0000 rate:0.0640 aloss:0.6976 eloss:-2.6373 aloss2:31.0657 exploreP:0.2269
Episode:318 meanR:56.9800 R:35.0000 rate:0.0700 aloss:0.7082 eloss:-5.1880 aloss2:29.1566 exploreP:0.2261
Episode:319 meanR:56.7400 R:49.0000 rate:0.0980 aloss:0.6864 eloss:-4.7379 aloss2:30.1063 exploreP:0.2251
Episode:320 meanR:56.7300 R:48.0000 rate:0.0960 aloss:0.6879 eloss:-4.6333 aloss2:30.7805 exploreP:0.2240
Episode:321 meanR:57.0000 R:59.0000 rate:0.1180 aloss:0.6951 eloss:-4.2215 aloss2:29.2413 exploreP:0.2228
Episode:322 meanR:57.4000 R:93.0000 rate:0.1860 aloss:0.6923 eloss:-4.9466 aloss2:29.3500 exploreP:0.2208
Episode:323 meanR:57.2500 R:53.0000 rate:0.1060 aloss:0.6816 eloss:-5.0235 aloss2:30.5014 exploreP:0.2197
Episode:324 meanR:57.0800 R:28.0000 rate:0.056

Episode:393 meanR:43.7700 R:29.0000 rate:0.0580 aloss:0.7230 eloss:-5.8296 aloss2:32.6273 exploreP:0.1659
Episode:394 meanR:43.8500 R:37.0000 rate:0.0740 aloss:0.7016 eloss:-5.5571 aloss2:32.3798 exploreP:0.1653
Episode:395 meanR:44.0200 R:45.0000 rate:0.0900 aloss:0.7045 eloss:-5.4016 aloss2:33.7429 exploreP:0.1646
Episode:396 meanR:43.9300 R:31.0000 rate:0.0620 aloss:0.7192 eloss:-5.4677 aloss2:32.6872 exploreP:0.1641
Episode:397 meanR:43.7800 R:54.0000 rate:0.1080 aloss:0.7083 eloss:-5.0664 aloss2:33.1157 exploreP:0.1633
Episode:398 meanR:43.7600 R:37.0000 rate:0.0740 aloss:0.7465 eloss:-5.4894 aloss2:33.4729 exploreP:0.1627
Episode:399 meanR:43.7400 R:39.0000 rate:0.0780 aloss:0.7264 eloss:-5.4190 aloss2:33.8777 exploreP:0.1621
Episode:400 meanR:43.9200 R:47.0000 rate:0.0940 aloss:0.7136 eloss:-5.0274 aloss2:33.5586 exploreP:0.1614
Episode:401 meanR:43.8300 R:33.0000 rate:0.0660 aloss:0.7383 eloss:-4.4496 aloss2:33.5844 exploreP:0.1609
Episode:402 meanR:44.0100 R:47.0000 rate:0.094

Episode:471 meanR:38.1700 R:36.0000 rate:0.0720 aloss:0.7232 eloss:-5.7739 aloss2:35.5743 exploreP:0.1262
Episode:472 meanR:38.2600 R:43.0000 rate:0.0860 aloss:0.7389 eloss:-5.2083 aloss2:36.3364 exploreP:0.1257
Episode:473 meanR:38.0700 R:31.0000 rate:0.0620 aloss:0.7054 eloss:-5.9128 aloss2:36.1070 exploreP:0.1253
Episode:474 meanR:37.9900 R:35.0000 rate:0.0700 aloss:0.7128 eloss:-4.9216 aloss2:35.1423 exploreP:0.1249
Episode:475 meanR:37.8800 R:46.0000 rate:0.0920 aloss:0.7137 eloss:-5.6006 aloss2:36.1632 exploreP:0.1244
Episode:476 meanR:37.6900 R:28.0000 rate:0.0560 aloss:0.6802 eloss:-5.8186 aloss2:36.4797 exploreP:0.1241
Episode:477 meanR:37.6300 R:30.0000 rate:0.0600 aloss:0.7002 eloss:-5.7011 aloss2:35.3139 exploreP:0.1237
Episode:478 meanR:37.7000 R:36.0000 rate:0.0720 aloss:0.7917 eloss:-4.0907 aloss2:36.3483 exploreP:0.1233
Episode:479 meanR:37.7000 R:31.0000 rate:0.0620 aloss:0.7336 eloss:-4.0940 aloss2:36.2775 exploreP:0.1230
Episode:480 meanR:37.7300 R:48.0000 rate:0.096

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
